<a href="https://colab.research.google.com/github/XuanTu2002/FinBERT-VCSenti/blob/main/FinBERT_VCSenti.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
# Bước 1: Cài đặt các thư viện cần thiết
!pip install transformers datasets evaluate accelerate -q

import pandas as pd
import numpy as np
import torch
import evaluate
from datasets import Dataset
from sklearn.model_selection import train_test_split
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding,
    pipeline
)

In [24]:
# Bước 2: Tải và chuẩn bị dữ liệu
# Tải dữ liệu từ Hugging Face Hub
try:
    from datasets import load_dataset
    full_dataset = load_dataset("atrost/financial_phrasebank")
    df = full_dataset['train'].to_pandas()
except Exception as e:
    print(f"Lỗi khi tải dữ liệu, vui lòng kiểm tra lại kết nối hoặc tên dataset. Lỗi: {e}")

# Tạo các ánh xạ giữa id và tên nhãn để dễ hiểu hơn
# 0: negative, 1: neutral, 2: positive
id2label = {0: "negative", 1: "neutral", 2: "positive"}
label2id = {"negative": 0, "neutral": 1, "positive": 2}

# Chuyển đổi nhãn số thành chuỗi để dễ đọc
df['label_name'] = df['label'].map(id2label)

# Tách dữ liệu thành tập huấn luyện và tập kiểm thử
train_df, eval_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])

# Chuyển đổi pandas DataFrame trở lại thành Dataset object của Hugging Face
train_dataset = Dataset.from_pandas(train_df)
eval_dataset = Dataset.from_pandas(eval_df)

print("Ví dụ dữ liệu huấn luyện:")
print(train_dataset[0])

Ví dụ dữ liệu huấn luyện:
{'sentence': 'Nokia has inaugurated its manufacturing plant near Chennai on 11 Mar 2006 .', 'label': 1, 'label_name': 'neutral', '__index_level_0__': 665}


In [25]:
# Bước 3: Tải Tokenizer và Model
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(
    checkpoint,
    num_labels=3,
    id2label=id2label,
    label2id=label2id
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [26]:
# Bước 4: Tokenize dữ liệu
def tokenize_function(examples):
    return tokenizer(examples["sentence"], truncation=True, padding=False)

tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_eval_dataset = eval_dataset.map(tokenize_function, batched=True)

# Dynamic padding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/2480 [00:00<?, ? examples/s]

Map:   0%|          | 0/620 [00:00<?, ? examples/s]

In [27]:
# Bước 5: Thiết lập và Huấn luyện
# Định nghĩa hàm tính toán các chỉ số đánh giá
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1) # Lấy nhãn có xác suất cao nhất

    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)
    f1 = f1_metric.compute(predictions=predictions, references=labels, average="weighted") # "weighted" F1-score is good for imbalanced datasets

    return {
        "accuracy": accuracy["accuracy"],
        "f1": f1["f1"],
    }

# Thiết lập các tham số huấn luyện (Training Arguments)
# Các tham số này được lấy cảm hứng từ paper FinBERT
# Lưu ý: batch_size=16 là con số an toàn cho Colab, paper gốc dùng batch_size=64
training_args = TrainingArguments(
    output_dir="finbert-sentiment-analysis",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=4,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
    report_to="none"
)

# Khởi tạo Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# Bắt đầu huấn luyện
print("Bắt đầu quá trình fine-tuning...")
trainer.train()
print("Quá trình huấn luyện đã hoàn tất!")

/tmp/ipython-input-1653967019.py:36: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Bắt đầu quá trình fine-tuning...


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.456889,0.817742,0.818541
2,No log,0.391016,0.854839,0.851877
3,No log,0.412417,0.858065,0.857044
4,0.377700,0.460124,0.854839,0.854946


Quá trình huấn luyện đã hoàn tất!


In [29]:
# Bước 6: Sử dụng mô hình đã huấn luyện
print("\nSử dụng pipeline để dự đoán câu mới:")
# Lấy model tốt nhất đã được lưu
best_model_checkpoint = trainer.state.best_model_checkpoint
pipe = pipeline("text-classification", model=best_model_checkpoint, device=0 if torch.cuda.is_available() else -1)

# Ví dụ dự đoán
new_sentences = [
    "The company announced a record-breaking quarter with huge profits.",
    "Despite the merger, the stock price has fallen.",
    "The report shows that the market is stable."
]

results = pipe(new_sentences)
for sentence, result in zip(new_sentences, results):
    print(f"'{sentence}' -> Dự đoán: {result['label']} (Điểm: {result['score']:.4f})")



Sử dụng pipeline để dự đoán câu mới:


Device set to use cuda:0


'The company announced a record-breaking quarter with huge profits.' -> Dự đoán: positive (Điểm: 0.9548)
'Despite the merger, the stock price has fallen.' -> Dự đoán: negative (Điểm: 0.7254)
'The report shows that the market is stable.' -> Dự đoán: positive (Điểm: 0.7772)
